SPV model

The following energy functional is used to derive the forces: 

E_i = kappa_A*(A_i-A0)^2 + kappa_P*(P_i-P0)^2 + Sum_<j interfaces of i> {J_ij *l_ij}

(I can explain properly if this is confusing)

First install the packages. The actual code is in **voronoi_model_periodic**

In [ ]:
from voronoi_model.voronoi_model_periodic import *
import numpy as np
import matplotlib.pyplot as plt

Set up your tissue object. This is "vor"

At the moment, the cell set-up is a bit messy. The initial condition should be a set of hexagons with randomly attributed 
cell types, conforming to the proportion "pE" (the proportion of ES cells here). 

In [ ]:
vor = Tissue()
vor.generate_cells(n_c=100) #(Approximate) number of cells. Have a play... I'll improve this later. 
vor.make_init(L=15,noise=0) #Set the length of the x and y axes. This should fill in hexagons with a fixed size. Crops the excess hexagons and re-adjusts the cell number. Very crude atm.

Set your interaction parameters. This is a bit like the W matrix much like in the lattice model. 

Note that contributions towards the adhesion term come from two places in the energy functional -- P0 and "J". 
Essentially you can view P0 as measuring the amount of homotypic adhesion, and then the J term sets the deviation from homotypic adhesion. 

In this case we have identical parameters for each cell type, but they repel each other (hence +ve W)

W_matrix is then mapped to a self.J matrix, which considers the interaction between every pair of cell ids. This is all done automatically

In [ ]:
alpha = 0.08
vor.set_interaction(W = alpha*np.array([[0, 1], [1, 0]]),pE=0.5)

p0 = 3.80 #shape index
vor.A0 = 0.86 
vor.P0 = p0*np.sqrt(vor.A0)
vor.v0 = 1e-2 #Instanteous velocity
vor.Dr = 0.01 #Reciprocal of persistence time
vor.kappa_A = 0.2 #Area term coeff. 
vor.kappa_P = 0.1 #Perim term coeff. 

Soft repulsion between cells is also coded in to prevent dodgy numerical issues. See the appendix of the PloS C.B. "Active Vertex" Paper

In [ ]:
vor.a = 0.3 #radius of inner soft shell
vor.k = 2 #Magnitude of repulsion

Set time parameters

In [ ]:
vor.set_t_span(dt=0.025,tfin=200)

Simulate!

In [ ]:
vor.simulate()

Animate the output

In [ ]:
vor.animate(n_frames=50)

Compute some stats and plot these 

In [ ]:
vor.get_self_self()
fig, ax = plt.subplots()
ax.plot(vor.self_self)
ax.set(xlabel="Time",ylabel="Fraction of self-self interactions")
fig.savefig("self_self.pdf")


ratio = 0.5
P0_eff = alpha*ratio/vor.kappa_P + vor.P0
p0_eff = P0_eff/np.sqrt(vor.A0)
print(p0_eff)

"""
Stat to measure q for each cell. 
And compare with neighbourhood and thus p0_eff
And MSD (short time-scale)
"""
